In [1]:
%env RERUN_NOTEBOOK_ASSET=serve-local

env: RERUN_NOTEBOOK_ASSET=serve-local


In [ ]:
from __future__ import annotations

import ipywidgets as widgets
from IPython.display import display

# Create the text input widget
text_input = widgets.Text(value="", placeholder="Type something here", description="Input:", disabled=False)

# Create the button widget
button = widgets.Button(
    description="Print Input",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to print input",
    icon="check",
)

# Create an output widget to display the results
output = widgets.Output()


# Define the button click event handler
def on_button_clicked(b):
    with output:
        # Clear previous output
        output.clear_output()
        # Print the current value of the text input
        print(f"You entered: {text_input.value}")


# Connect the button click event to the handler function
button.on_click(on_button_clicked)


from rerun_notebook import Viewer, ViewerCallbacks, SelectionItem

viewer = Viewer(
    url="https://app.rerun.io/version/nightly/examples/raw_mesh.rrd",
    panel_states={
        "blueprint": "hidden",
        "selection": "expanded",
        "time": "collapsed",
    },
)


class AnnotationCallbackHandler(ViewerCallbacks):
    def on_selection_change(self, selection):
        pass


viewer.register_callbacks(AnnotationCallbackHandler())

# Display all widgets
display(viewer)
display(text_input)
display(button)
display(output)

In [2]:
import ipywidgets as widgets
from IPython.display import display
import datetime
import math

import rerun as rr
from rerun.notebook import Viewer, ViewerCallbacks, SelectionItem


rr.init("\"/Users/gijsd/rerun/taco_play\"", recording_id="episode_0")

viewer = Viewer(width=960, height=720)

class RecordingMarker(ViewerCallbacks):
    def __init__(self, viewer):
        self.viewer = viewer
        self.markers = []
        self.timeline = "log_tick"
        self.time = 0
        self.entity_path = ""

        # Create widgets
        self.timestamp_button = widgets.Button(
            description="Add Timestamp", button_style="primary", tooltip="Mark current position in the recording"
        )

        self.comment_text = widgets.Text(
            placeholder="Optional comment", description="Comment:", style={"description_width": "initial"}
        )

        self.markers_output = widgets.Output(
            layout={
                "border": "1px solid #ddd",
                "padding": "10px",
                "margin": "10px 0px",
                "max_height": "300px",
                "overflow": "auto",
                "color": "white"
            }
        )

        self.status_output = widgets.Output(layout={"margin": "5px 0px"})

        # Set up button callback
        self.timestamp_button.on_click(self.add_marker)

        # Create layout
        self.widget = widgets.VBox([
            # widgets.HTML(f"{self.timeline}: {self.time}"),
            widgets.HBox([self.timestamp_button, self.comment_text]),
            self.markers_output,
            self.status_output,
        ])

    def on_time_update(self, time: float):
        self.time = math.floor(time)

    def on_timeline_change(self, timeline: str, time: float):
        self.timeline = timeline
        self.time = math.floor(time)

    def on_selection_change(self, selection: list[SelectionItem]):
        for s in selection:
            if s.kind == "entity_path":
                self.entity_path = s.entity_path


    def add_marker(self, button):
        """Add a new timestamp marker with optional comment."""
        comment = self.comment_text.value.strip()

        rr.set_time_sequence(self.timeline, self.time)
        rr.log(self.entity_path, rr.AnyValues(marker=(), comment=comment))

        with self.status_output:
            self.status_output.clear_output()
            print(f"✓ Marker added at {self.time}")
        self.comment_text.value = ""

    def display(self):
        """Display the widget."""
        display(self.widget)



DEV ENVIRONMENT DETECTED! Re-importing rerun from: /Users/gijsd/rerun/rerun/rerun_py/rerun_sdk


Serving rerun notebook assets at http://127.0.0.1:56908
import failed:  cannot import name 'StorageNodeClient' from 'rerun_bindings' (/Users/gijsd/rerun/rerun/rerun_py/rerun_bindings/__init__.py)


[2025-02-28T08:54:12Z DEBUG re_chunk::batcher] creating new chunk batcher config=ChunkBatcherConfig { flush_tick: 8ms, flush_num_bytes: 1048576, flush_num_rows: 18446744073709551615, chunk_max_rows_if_unsorted: 256, max_commands_in_flight: None, max_chunks_in_flight: None, hooks: BatcherHooks { on_insert: None, on_release: Some(ArrowRecordBatchReleaseCallback("0x50c76020220")) } }
[2025-02-28T08:54:12Z DEBUG re_sdk::recording_stream] setting recording info app_id="/Users/gijsd/rerun/taco_play" rec_id=episode_0
[2025-02-28T08:54:12Z DEBUG re_sdk::recording_stream] setting recording info app_id="/Users/gijsd/rerun/taco_play" rec_id=episode_0


In [3]:
marker = RecordingMarker(viewer)
viewer.register_callbacks(marker)

display(viewer)
marker.display()

Viewer()

In [4]:
rr.log_file_from_path("/Users/gijsd/rerun/taco_play_episide0.rrd")
# rr.log_file_from_path("/Users/gijsd/rerun/rerun_imu_signals_example.png")
# rr.log("hello", rr.TextLog("hey"))

[2025-02-28T08:54:17Z INFO  re_data_loader::load_file] Loading "/Users/gijsd/rerun/taco_play_episide0.rrd"…
[2025-02-28T08:54:17Z DEBUG re_data_loader::loader_rrd] Loading rrd data from filesystem… filepath="/Users/gijsd/rerun/taco_play_episide0.rrd" loader="rerun.data_loaders.Rrd"
[2025-02-28T08:54:17Z DEBUG re_data_loader::load_file] compatible loader found loader="rerun.data_loaders.Rrd" path="/Users/gijsd/rerun/taco_play_episide0.rrd"
[2025-02-28T08:54:17Z DEBUG re_log_encoding::decoder] Reached end of stream, iterator complete


In [8]:
rr.save("annotated_rrd.rrd")
rr.get_global_data_recording().flush(blocking=True)

[2025-02-28T08:56:28Z DEBUG re_log_encoding::file_sink] Saving file to "annotated_rrd.rrd"…
[2025-02-28T08:56:28Z DEBUG re_sdk::recording_stream] setting recording info app_id="/Users/gijsd/rerun/taco_play" rec_id=episode_0
[2025-02-28T08:56:28Z DEBUG re_log_encoding::file_sink] Log stream written to annotated_rrd.rrd


In [13]:
df = rr.dataframe.load_recording("annotated_rrd.rrd")


[2025-02-28T09:00:52Z DEBUG re_log_encoding::decoder] Reached end of stream, iterator complete


In [14]:
df.view(index="frame_index", contents="/observation.images.rgb_gripper").select().read_all()

pyarrow.Table
frame_index: int64
log_tick: int64
log_time: timestamp[ns]
/observation.images.rgb_gripper:comment: list<item: string>
  child 0, item: string
/observation.images.rgb_gripper:VideoTimestamp: list<item: int64>
  child 0, item: int64
/observation.images.rgb_gripper:Blob: list<item: list<item: uint8 not null>>
  child 0, item: list<item: uint8 not null>
      child 0, item: uint8 not null
----
frame_index: [[0],[1],...,[64],[65]]
log_tick: [[null],[null],...,[null],[null]]
log_time: [[null],[null],...,[null],[null]]
/observation.images.rgb_gripper:comment: [[null],[null],...,[null],[null]]
/observation.images.rgb_gripper:VideoTimestamp: [[[0]],[[66666667]],...,[[4266666667]],[[4333333333]]]
/observation.images.rgb_gripper:Blob: [[[[0,0,0,32,102,...,49,46,49,48,48]]],[[[0,0,0,32,102,...,49,46,49,48,48]]],...,[[[0,0,0,32,102,...,49,46,49,48,48]]],[[[0,0,0,32,102,...,49,46,49,48,48]]]]